In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/aptos2019/valid.csv
/kaggle/input/aptos2019/test.csv
/kaggle/input/aptos2019/train_1.csv
/kaggle/input/aptos2019/val_images/val_images/17f6c7072f61.png
/kaggle/input/aptos2019/val_images/val_images/0243404e8a00.png
/kaggle/input/aptos2019/val_images/val_images/0083ee8054ee.png
/kaggle/input/aptos2019/val_images/val_images/0ac436400db4.png
/kaggle/input/aptos2019/val_images/val_images/0d0a21fd354f.png
/kaggle/input/aptos2019/val_images/val_images/02da652c74b8.png
/kaggle/input/aptos2019/val_images/val_images/15e96e848b46.png
/kaggle/input/aptos2019/val_images/val_images/059bc89df7f4.png
/kaggle/input/aptos2019/val_images/val_images/04d029cfb612.png
/kaggle/input/aptos2019/val_images/val_images/08752092140d.png
/kaggle/input/aptos2019/val_images/val_images/0ef4c61dc056.png
/kaggle/input/aptos2019/val_images/val_images/1a7e3356b39c.png
/kaggle/input/aptos2019/val_images/val_images/06be1092a062.png
/kaggle/input/aptos2019/val_images/val_images/03a7f4a5786f.png
/kaggle/input/a

In [ ]:
import os, random, cv2, timm
import numpy as np, pandas as pd
import torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt, seaborn as sns

# ====================================================
# 1. --- CONFIG & SEED ---
# ====================================================
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BASE_PATH = '/kaggle/input/aptos2019'
TRAIN_CSV = os.path.join(BASE_PATH, 'train_1.csv')
VAL_CSV   = os.path.join(BASE_PATH, 'valid.csv')
TRAIN_DIR = os.path.join(BASE_PATH, 'train_images/train_images')
VAL_DIR   = os.path.join(BASE_PATH, 'val_images/val_images')

NUM_CLASSES = 5
IMG_SIZE_GLOBAL = 384
PATCH_SIZE = 224
PATCHES_PER_IMAGE = 16
BATCH_SIZE = 8
EPOCHS = 10
BASE_LR = 2e-4
WEIGHT_DECAY = 1e-4
AUX_LAMBDA = 0.2
LABEL_SMOOTH = 0.05
NUM_WORKERS = 2



In [ ]:
# ====================================================
# 2. --- TRANSFORMS ---
# ====================================================
global_tfms = A.Compose([
    A.Resize(IMG_SIZE_GLOBAL, IMG_SIZE_GLOBAL),
    A.CenterCrop(IMG_SIZE_GLOBAL, IMG_SIZE_GLOBAL),
    A.CLAHE(p=0.3),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2(),
])

patch_tfms = A.Compose([
    A.RandomResizedCrop(size=(PATCH_SIZE, PATCH_SIZE), scale=(0.4, 1.0), ratio=(0.75, 1.33)),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.3),
    A.CLAHE(p=0.25),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2(),
])

val_global_tfms = A.Compose([
    A.Resize(IMG_SIZE_GLOBAL, IMG_SIZE_GLOBAL),
    A.CenterCrop(IMG_SIZE_GLOBAL, IMG_SIZE_GLOBAL),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2(),
])

val_patch_tfms = A.Compose([
    A.CenterCrop(PATCH_SIZE, PATCH_SIZE),
    A.Normalize(mean=(0.485,0.456,0.406), std=(0.229,0.224,0.225)),
    ToTensorV2(),
])

In [ ]:
 #3. --- HELPER: LESION PATCH EXTRACTION ---
# ====================================================
def extract_lesion_patches(img, num_patches=PATCHES_PER_IMAGE, patch_size=PATCH_SIZE):
    """
    Extract patches centered on lesion-like regions (bright spots, hemorrhages).
    If fewer lesions found, fill the rest with random crops.
    """
    h, w, _ = img.shape
    lesion_patches = []

    # 1. Preprocess: grayscale + contrast enhance
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    gray = cv2.equalizeHist(gray)

    # 2. Threshold for bright regions (candidate lesions)
    _, th = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # 3. Find contours (possible lesions)
    cnts, _ = cv2.findContours(th, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # 4. Sample patches around lesion centroids
    for c in cnts[:num_patches]:
        x, y, w_box, h_box = cv2.boundingRect(c)
        cx, cy = x + w_box // 2, y + h_box // 2

        # crop around centroid
        x1 = max(0, cx - patch_size // 2)
        y1 = max(0, cy - patch_size // 2)
        x2 = min(w, x1 + patch_size)
        y2 = min(h, y1 + patch_size)

        patch = img[y1:y2, x1:x2]
        if patch.shape[0] == patch_size and patch.shape[1] == patch_size:
            lesion_patches.append(patch)

    # 5. If not enough lesion patches, add random ones
    while len(lesion_patches) < num_patches:
        x = np.random.randint(0, w - patch_size)
        y = np.random.randint(0, h - patch_size)
        lesion_patches.append(img[y:y+patch_size, x:x+patch_size])

    return lesion_patches[:num_patches]


In [ ]:
 #====================================================
# 4. --- DATASET ---
# ====================================================
class DRMilDataset(Dataset):
    def __init__(self, df: pd.DataFrame, image_dir: str, train: bool = True):
        self.df = df.reset_index(drop=True)
        self.image_dir = image_dir
        self.train = train

    def __len__(self):
        return len(self.df)

    def _read_rgb(self, path: str):
        img = cv2.imread(path)
        return cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    def __getitem__(self, idx: int):
        row = self.df.iloc[idx]
        img_name = row['id_code'] + '.png'
        y = int(row['diagnosis'])
        path = os.path.join(self.image_dir, img_name)
        img = self._read_rgb(path)

        # Global image
        g = (global_tfms if self.train else val_global_tfms)(image=img)['image']

        # Lesion-guided patches
        patches_raw = extract_lesion_patches(img, PATCHES_PER_IMAGE, PATCH_SIZE)
        tfm = patch_tfms if self.train else val_patch_tfms
        patches = torch.stack([tfm(image=p)['image'] for p in patches_raw], 0)

        return g, patches, y

In [ ]:
#====================================================
# 5. --- CORN TARGETS & LOSS ---
# ====================================================
def corn_targets(y: torch.Tensor, num_classes: int, label_smooth: float = LABEL_SMOOTH):
    B = y.size(0)
    k = num_classes - 1
    arange = torch.arange(k, device=y.device).unsqueeze(0).expand(B, -1)
    t = (y.unsqueeze(1) > arange).float()
    if label_smooth > 0:
        t = t * (1 - label_smooth) + 0.5 * label_smooth
    return t

class CornLoss(nn.Module):
    def __init__(self, num_classes: int):
        super().__init__()
        self.num_classes = num_classes
        self.bce = nn.BCEWithLogitsLoss(reduction='none')
    def forward(self, logits: torch.Tensor, y: torch.Tensor):
        t = corn_targets(y, self.num_classes)
        return self.bce(logits, t).mean()

@torch.no_grad()
def corn_logits_to_label(logits: torch.Tensor):
    probs = torch.sigmoid(logits)
    return (probs > 0.5).sum(dim=1)

# ====================================================
# 6. --- ATTENTION MIL & FUSION MODEL ---
# ====================================================
class GatedAttentionMIL(nn.Module):
    def __init__(self, in_dim: int, attn_dim: int = 256):
        super().__init__()
        self.V, self.U, self.w = nn.Linear(in_dim, attn_dim), nn.Linear(in_dim, attn_dim), nn.Linear(attn_dim, 1)
    def forward(self, H: torch.Tensor):
        Vh, Uh = torch.tanh(self.V(H)), torch.sigmoid(self.U(H))
        A = self.w(Vh * Uh).squeeze(-1)
        A = torch.softmax(A, dim=1)
        M = torch.sum(A.unsqueeze(-1) * H, dim=1)
        return M, A

class LesionAwareFusion(nn.Module):
    def __init__(self, num_classes=5, patch_backbone='resnet18', global_backbone='tf_efficientnetv2_s_in21ft1k'):
        super().__init__()

        # 1. Patch encoder
        self.patch_encoder = timm.create_model(patch_backbone, pretrained=True, num_classes=0)
        self.patch_dim = self.patch_encoder.num_features
        self.attn = GatedAttentionMIL(self.patch_dim, 256)

        # 2. Global encoder
        self.global_encoder = timm.create_model(global_backbone, pretrained=True, num_classes=0)
        self.global_dim = self.global_encoder.num_features

        # 3. Fusion
        fusion_dim = self.patch_dim + self.global_dim
        self.fusion_head = nn.Sequential(
            nn.Linear(fusion_dim, 768), nn.ReLU(), nn.Dropout(0.4),
            nn.Linear(768, 384), nn.ReLU(), nn.Dropout(0.3),
        )

        # 4. Heads
        self.main_head = nn.Linear(384, num_classes - 1)
        self.patch_head = nn.Linear(self.patch_dim, num_classes - 1)

    def forward(self, g_img: torch.Tensor, patches: torch.Tensor):
        B, N, C, H, W = patches.shape
        patches = patches.view(B * N, C, H, W)

        # Patch encoding + MIL attention
        P = self.patch_encoder(patches).view(B, N, -1)
        bag_emb, attn_w = self.attn(P)

        # Global encoding
        G = self.global_encoder(g_img)

        # Fusion
        fused = torch.cat([bag_emb, G], 1)
        z = self.fusion_head(fused)

        # Outputs
        main_logits = self.main_head(z)
        patch_logits = self.patch_head(P)

        return main_logits, patch_logits, attn_w


In [ ]:
#====================================================
# 7. --- TRAINING & VALIDATION ---
# ====================================================
def train_one_epoch(model, loader, optimizer, scaler, main_crit):
    model.train(); total_main = total_aux = total = 0
    for g, patches, y in loader:
        g, patches, y = g.to(device), patches.to(device), y.to(device)
        optimizer.zero_grad(set_to_none=True)
        with autocast():
            main_logits, patch_logits, attn_w = model(g, patches)
            main_loss = main_crit(main_logits, y)

            # Aux loss
            B, N, Km1 = patch_logits.shape
            y_flat = y.unsqueeze(1).repeat(1, N).view(-1)
            t = corn_targets(y_flat, NUM_CLASSES)
            bce = F.binary_cross_entropy_with_logits(patch_logits.view(B * N, Km1), t, reduction='none')
            per_patch = bce.mean(1).view(B, N)
            aux_loss = (attn_w * per_patch).sum(1).mean()

            loss = main_loss + AUX_LAMBDA * aux_loss
        scaler.scale(loss).backward(); scaler.step(optimizer); scaler.update()
        bs = g.size(0)
        total += bs; total_main += main_loss.item() * bs; total_aux += aux_loss.item() * bs
    return total_main/total, total_aux/total

@torch.no_grad()
def validate(model, loader, main_crit):
    model.eval(); total_loss=total=0; all_preds, all_labels=[], []
    for g, patches, y in loader:
        g, patches, y = g.to(device), patches.to(device), y.to(device)
        main_logits, _, _ = model(g, patches)
        loss = main_crit(main_logits, y)
        pred = corn_logits_to_label(main_logits)
        all_preds.extend(pred.cpu().numpy()); all_labels.extend(y.cpu().numpy())
        bs = g.size(0); total_loss += loss.item()*bs; total += bs
    return total_loss/total, np.array(all_labels), np.array(all_preds)

# ====================================================
# 8. --- MAIN ---
# ====================================================
if __name__ == '__main__':
    train_df, val_df = pd.read_csv(TRAIN_CSV), pd.read_csv(VAL_CSV)
    train_ds, val_ds = DRMilDataset(train_df, TRAIN_DIR, True), DRMilDataset(val_df, VAL_DIR, False)
    train_loader = DataLoader(train_ds, BATCH_SIZE, True, num_workers=NUM_WORKERS, pin_memory=True, drop_last=True)
    val_loader = DataLoader(val_ds, BATCH_SIZE, False, num_workers=NUM_WORKERS, pin_memory=True)

    model = LesionAwareFusion(NUM_CLASSES).to(device)
    main_crit = CornLoss(NUM_CLASSES)
    optimizer = torch.optim.AdamW(model.parameters(), lr=BASE_LR, weight_decay=WEIGHT_DECAY)
    scaler = GradScaler()

    best_val = 1e9
    for epoch in range(1, EPOCHS+1):
        main_l, aux_l = train_one_epoch(model, train_loader, optimizer, scaler, main_crit)
        val_l, y_true, y_pred = validate(model, val_loader, main_crit)
        print(f"Epoch {epoch} | train_main {main_l:.4f} | train_aux {aux_l:.4f} | val {val_l:.4f}")
        if val_l < best_val:
            best_val = val_l; torch.save({'model': model.state_dict()}, 'mil_fusion_best.pth')

    ckpt = torch.load('mil_fusion_best.pth', map_location=device)
    model.load_state_dict(ckpt['model'])
    val_l, y_true, y_pred = validate(model, val_loader, main_crit)
    print(classification_report(y_true, y_pred, zero_division=0))
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.show()

model.safetensors:   0%|          | 0.00/46.8M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/timm/models/_factory.py:126: UserWarning: Mapping deprecated model name tf_efficientnetv2_s_in21ft1k to current tf_efficientnetv2_s.in21k_ft_in1k.
  model = create_fn(


model.safetensors:   0%|          | 0.00/86.5M [00:00<?, ?B/s]

/tmp/ipykernel_36/2134951261.py:51: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_36/2134951261.py:9: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch 1 | train_main 0.3070 | train_aux 0.4407 | val 0.2876
Epoch 2 | train_main 0.2502 | train_aux 0.3861 | val 0.2592
Epoch 3 | train_main 0.2294 | train_aux 0.3635 | val 0.2392
Epoch 4 | train_main 0.2130 | train_aux 0.3632 | val 0.2293
